In [437]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from multiprocessing import Process, Queue
import os
import numpy as np
import seaborn as sns
import tensorflow as tf
import pandas as pd
from scipy.misc import imsave
import scipy.stats
import random
from sklearn.metrics import accuracy_score

from problog import get_evaluatable
from problog.program import PrologString, SimpleProgram
from problog.core import ProbLog
from problog.logic import Constant,Var,Term,AnnotatedDisjunction
from problog.learning import lfi

%matplotlib notebook

In [523]:
model = """
t(0.5)::gender.
t(0.25)::age_1;t(0.25)::age_2;t(0.25)::age_3;t(0.25)::age_4.
t(0.5)::smoke.
t(0.5)::alcohol.

t(0.33)::bmi_1;t(0.33)::bmi_2;t(0.33)::bmi_3.
t(0.5)::ldl.
t(0.33)::blood_pressure_1;t(0.33)::blood_pressure_2;t(0.33)::blood_pressure_3.

t(_)::blood_pressure_1:-gender, age_1, age_2, age_3, age_4, smoke, alcohol.

t(_)::blood_pressure_2:-gender, age_1, age_2, age_3, age_4, smoke, alcohol.

t(_)::blood_pressure_3:-gender, age_1, age_2, age_3, age_4, smoke, alcohol.

t(_)::ldl_t:-gender, age_1, age_2, age_3, age_4, smoke, alcohol.

t(_)::ldl_f:-gender, age_1, age_2, age_3, age_4, smoke, alcohol.

t(_)::bmi_1:-gender, age_1, age_2, age_3, age_4, smoke, alcohol.

t(_)::bmi_2:-gender, age_1, age_2, age_3, age_4, smoke, alcohol.

t(_)::bmi_3:-gender, age_1, age_2, age_3, age_4, smoke, alcohol.

t(_)::heart_d:-blood_pressure_1, blood_pressure_2, blood_pressure_3, ldl, bmi_1, bmi_2, bmi_3.

t(_)::glucose:-blood_pressure_1, blood_pressure_2, blood_pressure_3, ldl, bmi_1, bmi_2, bmi_3.
"""

In [524]:
gender = Term('gender')
age_1 = Term('age_1')
age_2 = Term('age_2')
age_3 = Term('age_3')
age_4 = Term('age_4')
smoke = Term('smoke')
alcohol = Term('alcohol')

bmi_1 = Term('bmi_1')
bmi_2 = Term('bmi_2')
bmi_3 = Term('bmi_3')
ldl = Term('ldl')
blood_pressure_1 = Term('blood_pressure_1')
blood_pressure_2 = Term('blood_pressure_2')
blood_pressure_3 = Term('blood_pressure_3')

heart_d = Term('heart_d')
glucose = Term('glucose')

In [525]:
examples = [
    [(gender, False), (age_1, True), (bmi_1, True)],
    [(gender, False), (age_2, True)]
]

In [529]:
#examples = [
#    [(gender_female, True), (age_2, True), (blood_pressure_1, True)],
#    [(gender_female, False), (age_2, False), (blood_pressure_2, False)]
#]
df= pd.read_pickle("data.pkl")
def convert_row(row):
    results = []
    
    if row["RIAGENDR"]==2.0:
        results += [(gender, False)]
    if row["RIAGENDR"]==1.0:
        results += [(gender, True)]
    if row["RIDAGEYR"]==0.0:
        results += [(age_1, True)]
        results += [(age_2, False)]
        results += [(age_3, False)]
        results += [(age_4, False)]
    if row["RIDAGEYR"]==1.0:
        results += [(age_1, False)]
        results += [(age_2, True)]
        results += [(age_3, False)]
        results += [(age_4, False)]
    if row["RIDAGEYR"]==2.0:
        results += [(age_1, False)]
        results += [(age_2, False)]
        results += [(age_3, True)]
        results += [(age_4, False)]
    if row["RIDAGEYR"]==3.0:
        results += [(age_1, False)]
        results += [(age_2, False)]
        results += [(age_3, False)]
        results += [(age_4, True)]
    if row["ALQ120Q"]==1.0:
        results += [(alcohol, True)]
    if row["ALQ120Q"]==0.0:
        results += [(alcohol, False)]
    if row["SMQ020"]==1.0:
        results += [(smoke, True)]
    if row["SMQ020"]==2.0:
        results += [(smoke, False)]
    if row["BPXDI1"]==0.0:
        results += [(blood_pressure_1, True)]
        results += [(blood_pressure_2, False)]
        results += [(blood_pressure_3, False)]
    if row["BPXDI1"]==1.0:
        results += [(blood_pressure_1, False)]
        results += [(blood_pressure_2, True)]
        results += [(blood_pressure_3, False)]
    if row["BPXDI1"]==2.0:
        results += [(blood_pressure_1, False)]
        results += [(blood_pressure_2, False)]
        results += [(blood_pressure_3, True)]
    if row["LBDLDL"]==0.0:
        results += [(ldl, True)]
    if row["LBDLDL"]==1.0:
        results += [(ldl, False)]
    if row["BMXBMI"]==0.0:
        results += [(bmi_1, True)]
        results += [(bmi_2, False)]
        results += [(bmi_3, False)]
    if row["BMXBMI"]==1.0:
        results += [(bmi_1, False)]
        results += [(bmi_2, True)]
        results += [(bmi_3, False)]
    if row["BMXBMI"]==2.0:
        results += [(bmi_1, False)]
        results += [(bmi_2, False)]
        results += [(bmi_3, True)]
    if row["HEARTDISEASE"]==True:
        results += [(heart_d, True)]
    if row["HEARTDISEASE"]==False:
        results += [(heart_d, False)]
    if row["LBXGLU"]==1.0:
        results += [(glucose, True)]
    if row["LBXGLU"]==0.0:
        results += [(glucose, False)]
    
    
    return results
#evidence = [convert_row(df.iloc[0]), convert_row(df.iloc[3])]
evidence = [convert_row(df.iloc[i]) for i in range(0,10)]
#evidence = [[(gender, True), (age_1, True), (bmi_1, True)],
# [(gender, False), (age_2, True)]]
evidence

[[(gender, False),
  (age_1, True),
  (age_2, False),
  (age_3, False),
  (age_4, False),
  (bmi_1, True),
  (bmi_2, False),
  (bmi_3, False)],
 [(gender, True),
  (age_1, False),
  (age_2, False),
  (age_3, False),
  (age_4, True),
  (alcohol, False),
  (smoke, False),
  (blood_pressure_1, True),
  (blood_pressure_2, False),
  (blood_pressure_3, False),
  (ldl, False),
  (bmi_1, True),
  (bmi_2, False),
  (bmi_3, False),
  (heart_d, False),
  (glucose, False)],
 [(gender, False),
  (age_1, True),
  (age_2, False),
  (age_3, False),
  (age_4, False),
  (blood_pressure_1, True),
  (blood_pressure_2, False),
  (blood_pressure_3, False),
  (ldl, True),
  (bmi_1, True),
  (bmi_2, False),
  (bmi_3, False)],
 [(gender, True),
  (age_1, True),
  (age_2, False),
  (age_3, False),
  (age_4, False)],
 [(gender, True),
  (age_1, False),
  (age_2, False),
  (age_3, True),
  (age_4, False),
  (alcohol, True),
  (smoke, True),
  (blood_pressure_1, False),
  (blood_pressure_2, True),
  (blood_pressur

In [530]:
score, weights, atoms, iteration, lfi_problem = lfi.run_lfi(PrologString(model), evidence)

In [531]:
print (lfi_problem.get_model())

0.5::gender.
0.4::age_1; 0.2::age_2; 0.3::age_3; 0.1::age_4.
0.749999999999998::smoke.
0.333333333367755::alcohol.
0.666666666666667::bmi_1; 0.222222222222222::bmi_2; 0.111111111111111::bmi_3.
0.571428571428571::ldl.
0.714285714285714::blood_pressure_1; 0.142857142857143::blood_pressure_2; 0.142857142857143::blood_pressure_3.
0.452790042549091::blood_pressure_1 :- gender, age_1, age_2, age_3, age_4, smoke, alcohol.
0.833332201859::blood_pressure_2 :- gender, age_1, age_2, age_3, age_4, smoke, alcohol.
0.436084455963535::blood_pressure_3 :- gender, age_1, age_2, age_3, age_4, smoke, alcohol.
0.59535845216385::ldl_t :- gender, age_1, age_2, age_3, age_4, smoke, alcohol.
0.448982220212909::ldl_f :- gender, age_1, age_2, age_3, age_4, smoke, alcohol.
0.42856015477693::bmi_1 :- gender, age_1, age_2, age_3, age_4, smoke, alcohol.
0.450799373240498::bmi_2 :- gender, age_1, age_2, age_3, age_4, smoke, alcohol.
0.714659342242772::bmi_3 :- gender, age_1, age_2, age_3, age_4, smoke, alcohol.
0.75